## Build AWS image recognition enviornment
This Python notebook creates an image recognition system on AWS as described in Ass3:
- Creation of a S3 bucket
- Creation of ECR registry
- Creation of a Lambda function using build docker image
- Creation of a S3 Notification Trigger for S3 Bucket

In [1]:
import src.aws.utils as aws
import subprocess

In [2]:
REGION = 'us-west-2'

### 1. Create a S3 Buckets and include necessery data

In [3]:
S3_IMAGE_BUCKET_NAME = 'image-bucket-11920555'

In [4]:
aws.create_s3_bucket(S3_IMAGE_BUCKET_NAME, REGION) # create bucket to store images

Bucket already created.


### 2. Ceate ECR Registry to deploy Lambda images on it

In [5]:
ECR_REPOSITORY_NAME = 'repository-11920555'

In [6]:
aws.create_ecr_repository(ECR_REPOSITORY_NAME, REGION)
aws.list_ecr_repositories(REGION) # list repositories

Repository repository-11920555 already exists.
Available ECR repositories:
Repository Name: e11920555-repository, URI: 737977651766.dkr.ecr.us-west-2.amazonaws.com/e11920555-repository
Repository Name: repository-11920555, URI: 737977651766.dkr.ecr.us-west-2.amazonaws.com/repository-11920555


### 3. Build & Push Lambda Image to Registry

In [7]:
LAMBDA_SRC_PATH = 'src/aws/lambda/image-detection'
IMAGE_NAME = "737977651766.dkr.ecr.us-west-2.amazonaws.com/repository-11920555:latest"

In [8]:
subprocess.run(f"docker build -t {IMAGE_NAME} {LAMBDA_SRC_PATH}", shell=True, check=True)
# HINT install aws cli locally
subprocess.run(f"aws ecr get-login-password --region {REGION} | docker login --username AWS --password-stdin {IMAGE_NAME}", shell=True, check=True)
subprocess.run(f"docker push {IMAGE_NAME}", shell=True, check=True)

Sending build context to Docker daemon  35.45MB
Step 1/7 : FROM public.ecr.aws/lambda/python:3.11
 ---> 608dcfd6dc29
Step 2/7 : RUN pip install boto3 opencv-python-headless numpy
 ---> Using cache
 ---> 1a70e81de666
Step 3/7 : COPY yolov3-tiny.weights /opt/yolov3-tiny.weights
 ---> Using cache
 ---> 9c7739269e5b
Step 4/7 : COPY yolov3-tiny.cfg /opt/yolov3-tiny.cfg
 ---> Using cache
 ---> b07ab28a0b3a
Step 5/7 : COPY coco.names /opt/coco.names
 ---> Using cache
 ---> 7e4025421764
Step 6/7 : COPY lambda_function.py ${LAMBDA_TASK_ROOT}
 ---> Using cache
 ---> f9586488165d
Step 7/7 : CMD ["lambda_function.lambda_handler"]
 ---> Using cache
 ---> d8ae1dbe1e03
Successfully built d8ae1dbe1e03
Successfully tagged 737977651766.dkr.ecr.us-west-2.amazonaws.com/repository-11920555:latest


DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

WARNING! Your password will be stored unencrypted in /home/taha/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
The push refers to repository [737977651766.dkr.ecr.us-west-2.amazonaws.com/repository-11920555]
016059a887b3: Preparing
cde2a5a33a2d: Preparing
685137418bc4: Preparing
0293a5a33d01: Preparing
c01ca353716e: Preparing
7c02691fbabf: Preparing
4a1b8fb65012: Preparing
e8e5c6e3f5d5: Preparing
dfa013bd87d6: Preparing
bdf365301f25: Preparing
c8203e562a8c: Preparing
7c02691fbabf: Waiting
4a1b8fb65012: Waiting
e8e5c6e3f5d5: Waiting
bdf365301f25: Waiting
dfa013bd87d6: Waiting
c8203e562a8c: Waiting
685137418bc4: Layer already exists
016059a887b3: Layer already exists
0293a5a33d01: Layer already exists
c01ca353716e: Layer already exists
cde2a5a33a2d: Layer already exists
e8e5c6e3f5d5: Layer already exists
dfa013bd87d6: Layer already exists
4a1b8fb65012: Layer already exists
7c02691fbabf: Layer already exists
bdf365301f25: Layer already exists
c8203e562a8c: Layer already exists
latest: digest: sha256:5a6d73f0aebb10017c95b8e3d5bc42f23b0e3960a3ffc4ad75b87082851b620f size: 2626


CompletedProcess(args='docker push 737977651766.dkr.ecr.us-west-2.amazonaws.com/repository-11920555:latest', returncode=0)

### 4. Create DynamoDB database

In [9]:
TABLE_NAME = 'ImageRecognitionResults'

In [10]:
aws.create_dynamodb_table(table_name=TABLE_NAME, region=REGION)

Table ImageRecognitionResults already exists. Deleting...
Table ImageRecognitionResults deleted successfully.
Creating ImageRecognitionResults table...
Table ImageRecognitionResults created successfully.


### 5. Create Lambda Function

#### a. Find appropiate IAM role we can use for the Lambda Function

In [11]:
aws.list_iam_roles()

Available IAM roles:
Role Name: AWSServiceRoleForAPIGateway, ARN: arn:aws:iam::737977651766:role/aws-service-role/ops.apigateway.amazonaws.com/AWSServiceRoleForAPIGateway
Role Name: AWSServiceRoleForAutoScaling, ARN: arn:aws:iam::737977651766:role/aws-service-role/autoscaling.amazonaws.com/AWSServiceRoleForAutoScaling
Role Name: AWSServiceRoleForAWSCloud9, ARN: arn:aws:iam::737977651766:role/aws-service-role/cloud9.amazonaws.com/AWSServiceRoleForAWSCloud9
Role Name: AWSServiceRoleForCloudWatchEvents, ARN: arn:aws:iam::737977651766:role/aws-service-role/events.amazonaws.com/AWSServiceRoleForCloudWatchEvents
Role Name: AWSServiceRoleForElastiCache, ARN: arn:aws:iam::737977651766:role/aws-service-role/elasticache.amazonaws.com/AWSServiceRoleForElastiCache
Role Name: AWSServiceRoleForElasticLoadBalancing, ARN: arn:aws:iam::737977651766:role/aws-service-role/elasticloadbalancing.amazonaws.com/AWSServiceRoleForElasticLoadBalancing
Role Name: AWSServiceRoleForOrganizations, ARN: arn:aws:iam::

In [12]:
aws.get_role_policies('LabRole')

[{'PolicyName': 'AmazonSSMManagedInstanceCore',
  'PolicyArn': 'arn:aws:iam::aws:policy/AmazonSSMManagedInstanceCore'},
 {'PolicyName': 'AmazonEKSClusterPolicy',
  'PolicyArn': 'arn:aws:iam::aws:policy/AmazonEKSClusterPolicy'},
 {'PolicyName': 'AmazonEC2ContainerRegistryReadOnly',
  'PolicyArn': 'arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryReadOnly'},
 {'PolicyName': 'AmazonEKSWorkerNodePolicy',
  'PolicyArn': 'arn:aws:iam::aws:policy/AmazonEKSWorkerNodePolicy'},
 {'PolicyName': 'c121298a2916915l6940471t1w737977651766-VocLabPolicy2-lxspusYwziKw',
  'PolicyArn': 'arn:aws:iam::737977651766:policy/c121298a2916915l6940471t1w737977651766-VocLabPolicy2-lxspusYwziKw'},
 {'PolicyName': 'c121298a2916915l6940471t1w737977651766-VocLabPolicy3-wnfau1xb1DEu',
  'PolicyArn': 'arn:aws:iam::737977651766:policy/c121298a2916915l6940471t1w737977651766-VocLabPolicy3-wnfau1xb1DEu'},
 {'PolicyName': 'c121298a2916915l6940471t1w737977651766-VocLabPolicy1-yWqKAGIVy1iB',
  'PolicyArn': 'arn:aws:iam::737977

In [13]:
LAB_ROLE_ARN = 'arn:aws:iam::737977651766:role/LabRole'

#### b. Create Lambda Function

In [14]:
LAMBDA_FUNCTION_NAME = 'ImageRecognitionFunction'

In [15]:
aws.delete_lambda_function(LAMBDA_FUNCTION_NAME)
aws.deploy_lambda_function(LAMBDA_FUNCTION_NAME, IMAGE_NAME, LAB_ROLE_ARN, 900, 1024, region=REGION)

Lambda function ImageRecognitionFunction deleted successfully.
Lambda function ImageRecognitionFunction created.


{'ResponseMetadata': {'RequestId': '9e90937f-daba-45a7-8746-e5f4a139e8ff',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Thu, 04 Jul 2024 19:13:18 GMT',
   'content-type': 'application/json',
   'content-length': '1201',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9e90937f-daba-45a7-8746-e5f4a139e8ff'},
  'RetryAttempts': 0},
 'FunctionName': 'ImageRecognitionFunction',
 'FunctionArn': 'arn:aws:lambda:us-west-2:737977651766:function:ImageRecognitionFunction',
 'Role': 'arn:aws:iam::737977651766:role/LabRole',
 'CodeSize': 0,
 'Description': '',
 'Timeout': 900,
 'MemorySize': 1024,
 'LastModified': '2024-07-04T19:13:17.745+0000',
 'CodeSha256': '5a6d73f0aebb10017c95b8e3d5bc42f23b0e3960a3ffc4ad75b87082851b620f',
 'Version': '11',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '1f74596a-17bd-49b5-a5f0-b07e71fa3122',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode': 'Creating',
 'PackageType': 'Image',
 'Architectures': 

In [16]:
LAMBDA_FUNCTION_ARN = 'arn:aws:lambda:us-west-2:737977651766:function:ImageRecognitionFunction'

### 5. Add S3 Bucket Trigger for Lambda

In [ ]:
aws.create_s3_lambda_trigger(S3_IMAGE_BUCKET_NAME, LAMBDA_FUNCTION_ARN)

### 6. Test Lambda function

In [18]:
aws.upload_file_to_s3(S3_IMAGE_BUCKET_NAME, 'input_folder/000000000016.jpg', 'test.jpg')

File input_folder/000000000016.jpg uploaded to image-bucket-11920555/test.jpg.


In [35]:
aws.get_recent_lambda_logs(LAMBDA_FUNCTION_NAME)

Log stream: 2024/07/04/[$LATEST]33cd36b1aa01437e82732652361be920
START RequestId: 242204b3-7254-464d-8e0f-91e50aa1b870 Version: $LATEST

{"id": "test.jpg", "time": 2.124650478363037, "objects": [{"label": "person", "accuracy": 0.8197214603424072}, {"label": "sports ball", "accuracy": 0.6291971802711487}]}

Results stored in DynamoDB successfully

END RequestId: 242204b3-7254-464d-8e0f-91e50aa1b870

REPORT RequestId: 242204b3-7254-464d-8e0f-91e50aa1b870	Duration: 2241.65 ms	Billed Duration: 3619 ms	Memory Size: 1024 MB	Max Memory Used: 452 MB	Init Duration: 1377.26 ms	



In [34]:
aws.read_dynamodb_table(TABLE_NAME)

[{'id': 'test.jpg',
  'time': Decimal('2.124650478363037'),
  'objects': [{'accuracy': Decimal('0.8197214603424072'), 'label': 'person'},
   {'accuracy': Decimal('0.6291971802711487'), 'label': 'sports ball'}]}]